In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import gc
import io
import os
from itertools import combinations

from IPython.display import display
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 200)

pd.reset_option('display.float_format')
pd.set_option('display.max_colwidth', None)  

from sitecustomize import ROOT # lib này được khởi tạo ban đầu dự án

import helpers.view as view
import helpers.EDA as EDA
import modules.multi as multi

importlib.reload(view)
importlib.reload(EDA)
importlib.reload(multi)

<module 'modules.multi' from 'd:\\Data Science\\modules\\multi.py'>

In [99]:
def cache_clear():
    for var in list(globals()):  
        if var not in _keep_vars and not var.startswith("_"):  
            del globals()[var]  

    gc.collect()
_keep_vars = set(globals().keys())  # lưu biến gốc

# POS CASH balance (số dư khoản vay tiêu dùng - point of sale (cash loans ; comsumer loans))

In [100]:
pos_balance = pd.read_pickle(ROOT + "/data/pkl/pos_cash_balance.p")

# feature engineering

In [101]:
pos_balance.loc[((pos_balance["NAME_CONTRACT_STATUS"]=='Active') & (pos_balance["CNT_INSTALMENT_FUTURE"]==0)), 'NAME_CONTRACT_STATUS'] = 'Completed' ####### sửa
pos_balance.loc[(["NAME_CONTRACT_STATUS"]=='Completed') & (pos_balance["CNT_INSTALMENT_FUTURE"]!=0), "CNT_INSTALMENT_FUTURE"] = "Active" ####### sửa

C:\Users\Admin\AppData\Local\Temp\ipykernel_24608\1150525613.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Active' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pos_balance.loc[(["NAME_CONTRACT_STATUS"]=='Completed') & (pos_balance["CNT_INSTALMENT_FUTURE"]!=0), "CNT_INSTALMENT_FUTURE"] = "Active" ####### sửa


In [102]:
pos_balance_0 = pos_balance[pos_balance['CNT_INSTALMENT_FUTURE']==0].copy()
pos_balance_1 = pos_balance[pos_balance['CNT_INSTALMENT_FUTURE']>0].copy()

In [103]:
# pos_balance_0['NAME_CONTRACT_STATUS'] = 'Completed' # thay bằng complete để drop duplicate, và tránh chia 0 trong pctchange ?

In [104]:
pos_balance_0.sort_values(['SK_ID_PREV', 'MONTHS_BALANCE'], ascending=[True, False], inplace=True)

In [105]:
pos_balance_0.drop_duplicates(['SK_ID_PREV', "NAME_CONTRACT_STATUS"], keep='last', inplace=True)

In [106]:
pos_balance = pd.concat([pos_balance_0, pos_balance_1], ignore_index=True)

In [107]:
pos_balance['CNT_INSTALMENT-s-CNT_INSTALMENT_FUTURE'] = pos_balance['CNT_INSTALMENT'] - pos_balance['CNT_INSTALMENT_FUTURE']
pos_balance['CNT_INSTALMENT_FUTURE-d-CNT_INSTALMENT'] = pos_balance['CNT_INSTALMENT_FUTURE'] / pos_balance['CNT_INSTALMENT']

pos_balance['SK_DPD-s-SK_DPD_DEF'] = pos_balance['SK_DPD'] - pos_balance['SK_DPD_DEF']

In [108]:
pos_balance.sort_values(['SK_ID_PREV', 'MONTHS_BALANCE'], inplace=True)
pos_balance.reset_index(drop=True, inplace=True)

In [109]:
cols = ['CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF']
df_list = []
for col in cols:
    df = multi.multi(col, pos_balance)
    df_list.append(df)
df = pd.concat(df_list, axis=1)

d:\Data Science\modules\multi.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ret_diff = ret_diff.rename(f'{c}_diff')


In [110]:
pos_balance = pd.concat([pos_balance, df], axis=1)

In [113]:
pos_balance.replace(np.inf, np.nan, inplace=True) 
pos_balance.replace(-np.inf, np.nan, inplace=True)

In [111]:
_keep_vars.update(["pos_balance"])

In [112]:
cache_clear()

In [115]:
pos_balance.to_pickle(ROOT + "/data/processed/f401_pos_balance.p")

In [116]:
pos_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,CNT_INSTALMENT-s-CNT_INSTALMENT_FUTURE,CNT_INSTALMENT_FUTURE-d-CNT_INSTALMENT,SK_DPD-s-SK_DPD_DEF,CNT_INSTALMENT_FUTURE_diff,CNT_INSTALMENT_FUTURE_pctchange,SK_DPD_diff,SK_DPD_pctchange,SK_DPD_DEF_diff,SK_DPD_DEF_pctchange
0,1000001,158271,-10,12.0,12.0,Active,0,0,0.0,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1000001,158271,-9,12.0,11.0,Active,0,0,1.0,0.916667,0,-1.0,-0.083333,0.0,NaN,0.0,NaN
2,1000001,158271,-8,2.0,0.0,Completed,0,0,2.0,0.0,0,-11.0,-1.0,0.0,NaN,0.0,NaN
3,1000002,101962,-54,4.0,4.0,Active,0,0,0.0,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1000002,101962,-53,6.0,3.0,Active,0,0,3.0,0.5,0,-1.0,-0.25,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520942,2843499,314148,-35,60.0,55.0,Active,0,0,5.0,0.916667,0,-1.0,-0.017857,0.0,NaN,0.0,NaN
9520943,2843499,314148,-34,60.0,54.0,Active,0,0,6.0,0.9,0,-1.0,-0.018182,0.0,NaN,0.0,NaN
9520944,2843499,314148,-33,60.0,52.0,Active,0,0,8.0,0.866667,0,-2.0,-0.037037,0.0,NaN,0.0,NaN
9520945,2843499,314148,-32,60.0,51.0,Active,0,0,9.0,0.85,0,-1.0,-0.019231,0.0,NaN,0.0,NaN
